In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import orekit
from pymsis import msis
import matplotlib.dates as mdates
import hrd_20250608.utilities_ds as u
from orekit.pyhelpers import setup_orekit_curdir
from os import path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import hrd_20250608.rope_class_hrd as rope 
import glob

In [2]:
# download_orekit_data_curdir( 'hrd_20250608/orekit-data.zip' )  # Comment this out once this file has already been downloaded for repeated runs
vm = orekit.initVM()
setup_orekit_curdir( './hrd_20250608/' )

print ( 'Java version:', vm.java_version )
print ( 'Orekit version:', orekit.VERSION )

# User inputs
forward_propagation_mins = 1  # forward propagation time (mins)
init_date = pd.to_datetime('2003-05-10 00:00:00')

latitude_values = [89.]  # degrees
local_time_values = [8.]  # hours
altitude_values = [358.]  # km
lla_array = np.vstack((latitude_values, local_time_values, altitude_values)).T.reshape((-1, 3))



Java version: 1.8.0_412
Orekit version: 13.0.1


In [3]:

# columns = [
#     'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
#     'accelerometer_density', 'dens_mean', 'flag_dens', 'flag_dens_mean'
# ]

# file_paths = sorted(glob.glob('hrd_20250608/champ_data/*.txt'))

# df_list = []

# for file_path in file_paths:
#     df = pd.read_csv(
#         file_path,
#         delim_whitespace=True,
#         comment='#',
#         names=columns,
#         on_bad_lines='skip'  # Skip lines that cannot be parsed
#     )
#     df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
#     df.set_index('datetime', inplace=True)
#     df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
#     df_list.append(df)

# champ_all = pd.concat(df_list)

# champ_all.sort_index(inplace=True)


columns = [
    'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
    'accelerometer_density', 'dens_mean', 'flag_dens', 'flag_dens_mean'
]

file_paths = sorted(glob.glob('hrd_20250608/grace_data/*.txt'))

df_list = []

for file_path in file_paths:
    df = pd.read_csv(
        file_path,
        delim_whitespace=True,
        comment='#',
        names=columns
    )
    df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
    df.set_index('datetime', inplace=True)
    df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
    df_list.append(df)

grace_all = pd.concat(df_list)

grace_all.sort_index(inplace=True)

# columns = [
#     'date', 'time', 'GPS', 'alt', 'lon', 'lat', 'lst', 'arglat',
#     'accelerometer_density'
# ]

# file_paths = sorted(glob.glob('hrd_20250608/swarm_data/SA_*.txt'))

# df_list = []

# for file_path in file_paths:
#     df = pd.read_csv(
#         file_path,
#         delim_whitespace=True,
#         comment='#',
#         names=columns
#     )
#     df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
#     df.set_index('datetime', inplace=True)
#     df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
#     df_list.append(df)

# swarma_all = pd.concat(df_list)

# swarma_all.sort_index(inplace=True)

# file_paths = sorted(glob.glob('hrd_20250608/swarm_data/SC_*.txt'))

# df_list = []

# for file_path in file_paths:
#     df = pd.read_csv(
#         file_path,
#         delim_whitespace=True,
#         comment='#',
#         names=columns
#     )
#     df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'])
#     df.set_index('datetime', inplace=True)
#     df.drop(columns=['date', 'time', 'GPS'], inplace=True)
    
#     df_list.append(df)

# swarmc_all = pd.concat(df_list)

# swarmc_all.sort_index(inplace=True)

In [45]:
msis_version = 2.1
omega = 45

def plot_densities(outputs_df, satellite_name = 'GRACE-FO', plot_name = '2023a'):
    fig, ax = plt.subplots(1, 1, figsize=(25, 10), sharex=False)

    # Subplot 1 — Density comparison
    ax.semilogy(outputs_df.datetime, outputs_df.accelerometer_density, label=f"{satellite_name} acceleromenter density", color='tab:blue', linewidth=2)
    ax.semilogy(outputs_df.datetime, outputs_df.msis, label="NRL-MSIS 2.1", color='green', linewidth=1.5)
    ax.semilogy(outputs_df.datetime, outputs_df.ensemble_density, label="Ensemble density", color='orange', linewidth=2)
    # ax.plot(outputs_df.datetime, outputs_df.debiased_ensemble_density, label="Debiased ensemble density", color='red', linewidth=1.)
    # ax.fill_between(outputs_df.datetime, outputs_df.debiased_ensemble_density - outputs_df.density_std, outputs_df.debiased_ensemble_density + outputs_df.density_std,
                    #    color='red', alpha=0.2, label="Ensemble confidence interval")
    ax.semilogy(outputs_df.datetime, outputs_df.dmd, label="DMD", color='black', linewidth=1.0)


    # Labels and legend
    ax.set_ylabel(r"$\rho$ (kg/m$^3$)", fontsize=14)
    ax.set_title(f"Neutral density comparison – {satellite_name} vs ROPE vs NRL-MSIS 2.1", fontsize=16)
    ax.legend(loc='upper left', bbox_to_anchor=(1.05, 1), borderaxespad=0.)
    

    ax12 = ax.twinx()
    ax12.plot(outputs_df.datetime, outputs_df.kp_prop, label="$K_p$", color='tab:red', linestyle='--', linewidth=1)
    ax12.set_ylabel(r"$K_p$", fontsize=14)

    ax.xaxis.set_major_locator(mdates.MinuteLocator(interval=90))
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d %H:%M'))

    for label in ax.get_xticklabels() + ax.get_yticklabels():
        label.set_rotation(25)
        label.set_fontweight('bold')
        label.set_fontsize(12)
    for label in ax.get_xticklabels():
        label.set_rotation(90)
    ax.grid(True, alpha=0.3)

    plt.xticks(rotation=25)
    # ax.set_ylim(0.15e-12, 5.2e-11)
    plt.tight_layout()
    # plt.savefig(f'./imgs/{satellite_name.lower()}_vs_rope_ensemble_{plot_name}.png', dpi=300, bbox_inches='tight')
    plt.show()

def build_output_data(start_date, end_date, interpolator, msis_version, dataset):
    
    subset = dataset.loc[start_date:end_date]
    
    timestamps = subset.index.values
    latitude_values = subset.lat.values # degrees
    local_time_values = subset.lst.values # hours
    longitude_values = subset.lon.values
    altitude_values = subset.alt.values/1000. # km
    lla_array = np.vstack((latitude_values, local_time_values, altitude_values)).T.reshape((-1, 3))

    all_models, dmd_density, ensemble_density, density_std = interpolator.interpolate_mins(timestamps, lla_array)

    result = msis.calculate(
        timestamps,
        longitude_values,
        latitude_values,
        altitude_values, 
        geomagnetic_activity=-1,
        version = msis_version
    )

    msis_rho = result[:, 0]
    accelerometer_density = subset["accelerometer_density"].values

    # t1 = interpolator.data.interval_hourly_drivers[1, :]
    # t2 = interpolator.data.interval_hourly_drivers[2, :]
    # t3 = interpolator.data.interval_hourly_drivers[3, :]
    # t4 = interpolator.data.interval_hourly_drivers[4, :]
    # kp = interpolator.data.interval_hourly_drivers[6, :]
    # f10 = interpolator.data.interval_hourly_drivers[5, :]
    # hourly_time_series = interpolator.data.hourly_date_series

    # print(hourly_time_series.shape, kp.shape, f10.shape)
    # print(hourly_time_series.min(), hourly_time_series.max())

    # hourly_drivers_df = pd.DataFrame({
    # 'datetime': pd.to_datetime(hourly_time_series),
    # 'f10': f10,
    # 'kp': kp, 't1': t1, 't2': t2, 't3': t3, 't4': t4})
    # hourly_drivers_df['datetime'] = pd.to_datetime(hourly_drivers_df['datetime'])

    # outputs_df = pd.merge(
    #     pd.DataFrame(timestamps, columns=['datetime']),
    #     on='datetime',
    #     how='left'
    # )
    # print('date_series, interval_interpolated_drivers[5, :]')
    # print(interpolator.data.date_series.shape, interpolator.data.interval_interpolated_drivers[5, :].shape)
    # interpolated_outputs = pd.DataFrame({'datetime': interpolator.data.date_series, 
    #     'f10_prop': interpolator.data.interval_interpolated_drivers[5, :], 
    #     'kp_prop': interpolator.data.interval_interpolated_drivers[6, :]})
    
    # outputs_df = pd.merge(
    #     outputs_df,
    #     on='datetime',
    #     how='left'
    # )

    # print(interpolator.data.interval_interpolated_drivers.shape, interpolator.data.date_series.shape, outputs_df.shape)

    outputs_df.sort_values('datetime', inplace=True)
    outputs_df.ffill(inplace=True)
    outputs_df.reset_index(drop=True, inplace=True)
    outputs_df['lst'] = local_time_values
    outputs_df['lat'] = latitude_values
    outputs_df['ensemble_density'] = ensemble_density
    outputs_df['density_std'] = density_std
    outputs_df['accelerometer_density'] = accelerometer_density
    outputs_df['msis'] = msis_rho
    outputs_df['dmd'] = dmd_density
    

    return timestamps, accelerometer_density, ensemble_density, dmd_density, \
        density_std, msis_rho, outputs_df, latitude_values, local_time_values


# def run_demo(start_date, end_date, rope_density, msis_version, dataset, bias_propagation):

#     timestamps, accelerometer_density, ensemble_density, dmd_density, \
#         density_std, msis_rho, outputs_df, latitudes, lst_values = \
#             build_output_data(start_date, end_date, rope_density, msis_version, dataset)
    
#     return outputs_df

def run_model(start_date, end_date, rope_density, msis_version, dataset):

    timestamps, accelerometer_density, ensemble_density, dmd_density, \
        density_std, msis_rho, outputs_df, latitudes, lst_values = \
            build_output_data(start_date, end_date, rope_density, msis_version, dataset)
    
    return outputs_df

# Latest Ridge parameters are [1., 1000, 10000, 100000]
for alpha_ridge in [1]:
    selected_bf_dict = {
            'poly': 1000,
            # 'poly17': 500,
            # 'poly12': 1000,
            # 'poly13': 10,
            # 'poly135': 500, 
            # 'poly1357': 10, 
            # 'poly_sincos4': 100, 
            # 'poly_sincos7': alpha_ridge,
            # 'poly_exp1': alpha_ridge,
            # 'poly_exp2': alpha_ridge,
            # 'poly_exp12': alpha_ridge,
            # 'poly_exp22': alpha_ridge
        }
    # for basis in all_bf_dict.keys():
    #     for alpha_ridge in [1, 5, 10, 100, 1000, 10000, 100000]:

    # selected_bf_dict = {
    #         basis: alpha_ridge
    #     }

    lst_bias = 0. #-2.0 is good
    alt_bias = 0.#40.
    delta_rho_ic = 6

    lt_low = 0
    lt_high = 23.66666667

    lat_low = -87.5
    lat_high = 87.5

    alt_low = 100
    alt_high = 980

    # #Check if start and end dates are compatible with forward_propagation


    # # for bp in np.arange(1, 20):
    # bias_propagation = 1


    start_date = "2003-10-28 00:00:00"
    end_date = "2003-10-28 00:10:00"
    forward_propagation_mins = 1
    satellite_name = 'GRACE'
    plot_name = '2003'
    dataset = grace_all


    sindy = rope.rope_propagator(selected_bf_dict = selected_bf_dict, delta_rho_ic = delta_rho_ic)
    sindy.propagate_models_mins(init_date = start_date, forward_propagation = forward_propagation_mins)
    rope_density = rope.rope_data_interpolator( data = sindy)

    outputs_df = run_model(start_date, end_date, rope_density, msis_version, dataset)
    plot_densities(outputs_df, satellite_name = satellite_name, plot_name = plot_name)


System is propagating from 2003-10-28 00:00:00 for 10 mins
> <string>(55)interpolate_mins()

[[[[-5.60284116e-06  6.01518695e-05 -1.80643074e-05 ... -7.55408255e-05
     1.37022726e-04  8.31663128e-05]
   [ 1.45568411e-04 -8.07651610e-04  8.51206642e-05 ...  4.00385616e-04
    -2.95966398e-03 -1.01222133e-03]
   [ 2.67320545e-04 -7.21919234e-04 -5.49086362e-05 ... -2.33597611e-03
    -4.98542935e-03 -1.49623503e-03]
   ...
   [ 2.86358525e-03  8.25350080e-03  3.04733240e-03 ... -8.08340497e-04
     9.32948256e-04 -4.99918638e-03]
   [ 2.81610969e-03  8.29092413e-03  3.13607254e-03 ... -5.63177164e-04
     7.41791038e-04 -5.35058090e-03]
   [ 2.78155692e-03  8.30184110e-03  3.19022802e-03 ... -3.66406457e-04
     6.14510442e-04 -5.56436135e-03]]

  [[-5.60966600e-06  6.07384245e-05 -2.54861370e-05 ... -9.23796470e-05
     1.33833237e-04  1.22864643e-04]
   [ 1.47902916e-04 -7.82733492e-04  8.36591207e-05 ...  3.26097943e-04
    -2.67071230e-03 -1.60250196e-03]
   [ 2.71186698e-04 -6.969

In [5]:
# Save outputs_df to an Excel sheet
excel_file_path = 'output_data.xlsx'
outputs_df.to_excel(excel_file_path, index=False)

In [39]:
outputs_df

,datetime,f10,kp,t1,t2,t3,t4,f10_prop,kp_prop,lst,lat,ensemble_density,density_std,accelerometer_density,msis,dmd
0,2003-10-28 00:00:00,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.258977,2.868816,16.099,43.873855,NaN,NaN,1.463398e-12,2.204828e-12,NaN
1,2003-10-28 00:00:30,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.258977,2.868816,16.104,45.783317,NaN,NaN,1.415334e-12,2.166696e-12,NaN
2,2003-10-28 00:01:30,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.258977,2.868816,16.113,49.600603,NaN,NaN,1.346600e-12,2.088184e-12,NaN
3,2003-10-28 00:02:00,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.282083,2.890309,16.119,51.508424,NaN,NaN,1.335730e-12,2.047980e-12,NaN
4,2003-10-28 00:03:00,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.293635,2.901056,16.131,55.322430,NaN,NaN,1.303341e-12,1.966060e-12,NaN
5,2003-10-28 00:03:30,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.293635,2.901056,16.138,57.228620,NaN,NaN,1.264157e-12,1.924652e-12,NaN
6,2003-10-28 00:04:00,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.305188,2.911803,16.146,59.134272,NaN,NaN,1.222230e-12,1.883132e-12,NaN
7,2003-10-28 00:04:30,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.305188,2.911803,16.155,61.039389,NaN,NaN,1.185145e-12,1.841736e-12,NaN
8,2003-10-28 00:05:00,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.316741,2.922550,16.165,62.943970,NaN,NaN,1.176989e-12,1.800667e-12,NaN
9,2003-10-28 00:05:30,274.4,3.0,1.0,0.0,0.448906,-0.893579,274.316741,2.922550,16.176,64.848009,NaN,NaN,1.171879e-12,1.760183e-12,NaN


In [43]:
sindy.z_dict['dmd']

array([[264.26869188, 264.29254472, 264.31639592, 264.34024558,
        264.36409379, 264.38794062, 264.41178617, 264.43563055,
        264.45947388, 264.48331625, 264.50715773, 264.53099843,
        264.55483843, 264.57867783, 264.60251673, 264.62635522,
        264.65019339, 264.67403134, 264.69786915, 264.72170694,
        264.74554478, 264.76938278, 264.79322103, 264.81705961,
        264.84089863, 264.86473818, 264.88857836, 264.91241925,
        264.93626095, 264.96010355, 264.98394716, 265.00779185,
        265.03163773, 265.05548488, 265.07933341, 265.1031834 ,
        265.12703495, 265.15088815, 265.17474309, 265.19859987,
        265.22245858, 265.24631932, 265.27018216, 265.29404722,
        265.31791458, 265.34178433, 265.36565657, 265.38953139,
        265.41340888, 265.43728913, 265.46117223, 265.48505829,
        265.50894739, 265.53283961, 265.55673507, 265.58063383,
        265.60453601, 265.62844169, 265.65235095, 265.6762639 ],
       [ 11.82562274,  11.83339306,  11